In [1]:
# ONLY FOR GOOGLE COLAB
from google.colab import drive
import os
drive.mount('/content/drive')
WORKING_DIRECTORY = '/content/drive/My Drive/hack_october/Dataset'

os.chdir(WORKING_DIRECTORY)

Mounted at /content/drive


In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np

In [3]:
features = pd.read_csv("augmented.csv")
features = features.drop(['Unnamed: 0'], axis=1)

expert_decisions = pd.read_csv("new_OpenPart.csv")
expert_decisions = expert_decisions.drop(['Unnamed: 0'], axis=1)
expert_decisions["Case"] = expert_decisions["Case"].apply(lambda x: '.'.join(x.split(".")[:-1]))
expert_decisions.columns = ["file_name", "sample_1", "sample_2", "sample_3"]

In [4]:
# expert_decisions = expert_decisions.melt(id_vars=['file_name'], var_name='user_name', value_name='mark')
# data = pd.merge(features, expert_decisions)

In [15]:
data = features

In [16]:
for col in data.columns:
    if "shape" in col:
        data[col] = data[col].apply(lambda x: 1 if x == "circle" else 0)

In [17]:
data = data.sort_values("file_name")

In [18]:
data = data.reset_index(drop=True)

In [19]:
data

,file_name,user_name,expert_xcenter_0,expert_ycenter_0,expert_rhorizontal_0,expert_rvertical_0,expert_shape_0,expert_xcenter_1,expert_ycenter_1,expert_rhorizontal_1,expert_rvertical_1,expert_shape_1,expert_xcenter_2,expert_ycenter_2,expert_rhorizontal_2,expert_rvertical_2,expert_shape_2,expert_xcenter_3,expert_ycenter_3,expert_rhorizontal_3,expert_rvertical_3,expert_shape_3,expert_xcenter_4,expert_ycenter_4,expert_rhorizontal_4,expert_rvertical_4,expert_shape_4,expert_xcenter_5,expert_ycenter_5,expert_rhorizontal_5,expert_rvertical_5,expert_shape_5,expert_xcenter_6,expert_ycenter_6,expert_rhorizontal_6,expert_rvertical_6,expert_shape_6,expert_xcenter_7,expert_ycenter_7,expert_rhorizontal_7,...,ycenter_17,rhorizontal_17,rvertical_17,shape_17,xcenter_18,ycenter_18,rhorizontal_18,rvertical_18,shape_18,xcenter_19,ycenter_19,rhorizontal_19,rvertical_19,shape_19,xcenter_20,ycenter_20,rhorizontal_20,rvertical_20,shape_20,xcenter_21,ycenter_21,rhorizontal_21,rvertical_21,shape_21,xcenter_22,ycenter_22,rhorizontal_22,rvertical_22,shape_22,iou,iou_intersections,center,ml_count,expert_count,expert_intersect_area,expert_union_area,expert_area,ml_intersect_area,ml_union_area,ml_area
0,00000150_002,sample_1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,...,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,0.351411,0.351411,0.000007,1.0,1.0,0.0,2729.0,2729.0,0.0,959.0,959.0
1,00000150_002,sample_2,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,...,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,0.306655,0.306655,0.000588,1.0,1.0,0.0,2729.0,2729.0,0.0,7540.0,7540.0
2,00000150_002,sample_3,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,...,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,0.069099,0.069099,0.001943,1.0,1.0,0.0,2729.0,2729.0,0.0,39494.0,39494.0
3,00000150_002(1.2),sample_1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,...,580.974093,15.032815,14.148532,1,643.758204,580.974093,15.032815,14.148532,1,643.758204,580.974093,15.032815,14.148532,1,643.758204,580.974093,15.032815,14.148532,1,643.758204,580.974093,15.032815,14.148532,1,643.758204,580.974093,15.032815,14.148532,1,0.377328,0.377328,0.000005,1.0,1.0,0.0,1879.0,1879.0,0.0,709.0,709.0
4,00000150_002(1.2),sample_2,642.8

In [20]:
data.to_csv('std_data_without_marks.csv')

In [9]:
y = data['mark']

In [10]:
X = data.drop(['file_name', 'user_name', 'mark'], axis=1)

In [11]:
X = 

,expert_xcenter_0,expert_ycenter_0,expert_rhorizontal_0,expert_rvertical_0,expert_shape_0,expert_xcenter_1,expert_ycenter_1,expert_rhorizontal_1,expert_rvertical_1,expert_shape_1,expert_xcenter_2,expert_ycenter_2,expert_rhorizontal_2,expert_rvertical_2,expert_shape_2,expert_xcenter_3,expert_ycenter_3,expert_rhorizontal_3,expert_rvertical_3,expert_shape_3,expert_xcenter_4,expert_ycenter_4,expert_rhorizontal_4,expert_rvertical_4,expert_shape_4,expert_xcenter_5,expert_ycenter_5,expert_rhorizontal_5,expert_rvertical_5,expert_shape_5,expert_xcenter_6,expert_ycenter_6,expert_rhorizontal_6,expert_rvertical_6,expert_shape_6,expert_xcenter_7,expert_ycenter_7,expert_rhorizontal_7,expert_rvertical_7,expert_shape_7,...,ycenter_17,rhorizontal_17,rvertical_17,shape_17,xcenter_18,ycenter_18,rhorizontal_18,rvertical_18,shape_18,xcenter_19,ycenter_19,rhorizontal_19,rvertical_19,shape_19,xcenter_20,ycenter_20,rhorizontal_20,rvertical_20,shape_20,xcenter_21,ycenter_21,rhorizontal_21,rvertical_21,shape_21,xcenter_22,ycenter_22,rhorizontal_22,rvertical_22,shape_22,iou,iou_intersections,center,ml_count,expert_count,expert_intersect_area,expert_union_area,expert_area,ml_intersect_area,ml_union_area,ml_area
0,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,...,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,772.509845,697.168912,18.039378,16.978238,1,0.351411,0.351411,0.000007,1.0,1.0,0.0,2729.0,2729.0,0.0,959.0,959.0
1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,...,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,737.882353,690.389140,64.482655,28.573152,0,0.306655,0.306655,0.000588,1.0,1.0,0.0,2729.0,2729.0,0.0,7540.0,7540.0
2,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,771.431635,700.739946,30.198391,28.139410,1,...,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,709.214815,686.459259,107.817989,90.480423,0,0.069099,0.069099,0.001943,1.0,1.0,0.0,2729.0,2729.0,0.0,39494.0,39494.0
3,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.949955,25.165326,23.449508,1,...,572.049383,89.848325,75.400353,0,591.012346,572.049383,89.848325,75.400353,0,591.012346,572.049383,89.848325,75.400353,0,591.012346,572.049383,89.848325,75.400353,0,591.012346,572.049383,89.848325,75.400353,0,591.012346,572.049383,89.848325,75.400353,0,0.068677,0.068677,0.001350,1.0,1.0,0.0,1879.0,1879.0,0.0,27360.0,27360.0
4,642.859696,583.949955,25.165326,23.449508,1,642.859696,583.

In [12]:
train_test_split = 0.2
test_size = int(len(X) * train_test_split)
X_train, X_test = X[test_size:], X[:test_size]
y_train, y_test = y[test_size:], y[:test_size]

In [13]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
from sklearn.externals import joblib
joblib.dump(clf, 'lgb.pkl')


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['lgb.pkl']

In [32]:
y_pred=clf.predict_proba(X_test)

In [ ]:
y_pred

In [31]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('L1 Model accuracy score: {0:0.4f}'.format(calculate_l1(y_test, y_pred)))

LightGBM Model accuracy score: 0.3846
L1 Model accuracy score: 124.0000


In [14]:
def calculate_l1(y_test, y_pred):
    l1 = 0
    for i in range(len(y_test)): 
        l1 += abs(y_test[i] - y_pred[i])
    return l1